##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guia inicial de TensorFlow 2.0 para expertos

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver en TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver codigo en GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Descargar notebook</a>
  </td>
</table>

Note: Nuestra comunidad de Tensorflow ha traducido estos documentos. Como las traducciones de la comunidad
son basados en el "mejor esfuerzo", no hay ninguna garantia que esta sea un reflejo preciso y actual 
de la [Documentacion Oficial en Ingles](https://www.tensorflow.org/?hl=en).
Si tienen sugerencias sobre como mejorar esta traduccion, por favor envian un "Pull request"
al siguiente repositorio [tensorflow/docs](https://github.com/tensorflow/docs).
Para ofrecerse como voluntario o hacer revision de las traducciones de la Comunidad
por favor contacten al siguiente grupo [docs@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Este es un notebook de Google Colaboratory. Los programas de Python se executan directamente en tu navegador —una gran manera de aprender y utilizar TensorFlow. Para poder seguir este tutorial, ejecuta este notebook en Google Colab presionando el boton en la parte superior de esta pagina.

En Colab, selecciona "connect to a Python runtime": En la parte superior derecha de la barra de menus selecciona: CONNECT.
Para ejecutar todas las celdas de este notebook: Selecciona Runtime > Run all.

Descarga e installa el paquete TensorFlow 2.0 version. 

Importa TensorFlow en tu programa:

Import TensorFlow into your program:

In [ ]:
import pandas as pd
from google.colab import drive 

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pickle
import numpy as np

# open a file, where you stored the pickled data
file = open('gdrive/My Drive/Colab Notebooks/x_traj_sel.pkl', 'rb')

# dump information to that file
x_data = pickle.load(file)

# close the file
file.close()


In [ ]:
# open a file, where you stored the pickled data
file = open('gdrive/My Drive/Colab Notebooks/y_traj_sel.pkl', 'rb')

# dump information to that file
y_data = pickle.load(file)

# close the file
file.close()

In [ ]:
print(type(y_data))

<class 'numpy.ndarray'>


In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [ ]:
print(len(x_data))

18447


In [ ]:
print(18447*0.75)

13835.25


In [ ]:
x_train = x_data[:14000]
x_test = x_data[14000:]

In [ ]:
y_train = y_data[:14000]
y_test = y_data[14000:]


In [ ]:
print(y_test.shape)

(4447,)


Utiliza `tf.data` to separar por lotes y mezclar el conjunto de datos:

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(1000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(1.),
])


In [ ]:
model = tf.keras.Sequential(
    [
    data_augmentation,
    layers.Conv2D(64, 3, padding='same', activation="relu",input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2), strides=2),

    layers.Conv2D(128, 3, padding='same', activation="relu"),
    layers.MaxPooling2D((2, 2), strides=2),

    layers.Flatten(),
    layers.Dense(150, activation="relu"),
    #layers.Dropout(0.2),
    layers.Dense(7, activation="softmax")
]
)

Construye el modelo `tf.keras` utilizando la API de Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing):

Escoge un optimizador y una funcion de perdida para el entrenamiento de tu modelo:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

Escoge metricas para medir la perdida y exactitud del modelo.
Estas metricas acumulan los valores cada epoch y despues imprimen el resultado total.

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Utiliza `tf.GradientTape` para entrenar el modelo.

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Prueba el modelo:

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reinicia las metricas para el siguiente epoch.
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Perdida: 0.8147780895233154, Exactitud: 68.79285430908203, Perdida de prueba: 0.577229380607605, Exactitud de prueba: 76.680908203125
Epoch 2, Perdida: 0.5668789148330688, Exactitud: 77.42857360839844, Perdida de prueba: 0.559723973274231, Exactitud de prueba: 77.73780059814453
Epoch 3, Perdida: 0.5401189923286438, Exactitud: 78.40714263916016, Perdida de prueba: 0.5241925120353699, Exactitud de prueba: 78.86215209960938
Epoch 4, Perdida: 0.5040915608406067, Exactitud: 79.9000015258789, Perdida de prueba: 0.5260542035102844, Exactitud de prueba: 78.70474243164062
Epoch 5, Perdida: 0.4922063946723938, Exactitud: 79.97857666015625, Perdida de prueba: 0.5403757095336914, Exactitud de prueba: 78.18754577636719
Epoch 6, Perdida: 0.47856366634368896, Exactitud: 80.94285583496094, Perdida de prueba: 0.48844271898269653, Exactitud de prueba: 80.14391326904297
Epoch 7, Perdida: 0.46929723024368286, Exactitud: 81.1500015258789, Perdida de prueba: 0.5120519995689392, Exactitud de prueba:

El model de clasificacion de images fue entrenado y alcanzo una exactitud de ~98% en este conjunto de datos. Para aprender mas, lee los [tutoriales de TensorFlow](https://www.tensorflow.org/tutorials).